# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f42c0e0d1f0>
├── 'a' --> tensor([[ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418]])
└── 'x' --> <FastTreeValue 0x7f42c0e0df10>
    └── 'c' --> tensor([[-0.1107,  0.4488,  1.0056, -1.7518],
                        [-0.8434,  1.5024, -0.4146,  0.1262],
                        [-1.4129,  1.2946, -0.5745, -0.7119]])

In [4]:
t.a

tensor([[ 0.6537,  0.6462, -1.1670],
        [-0.1494, -0.1912,  0.7418]])

In [5]:
%timeit t.a

65 ns ± 0.0336 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f42c0e0d1f0>
├── 'a' --> tensor([[ 1.4286, -1.1858,  0.4374],
│                   [ 0.2254,  0.9186,  0.5257]])
└── 'x' --> <FastTreeValue 0x7f42c0e0df10>
    └── 'c' --> tensor([[-0.1107,  0.4488,  1.0056, -1.7518],
                        [-0.8434,  1.5024, -0.4146,  0.1262],
                        [-1.4129,  1.2946, -0.5745, -0.7119]])

In [7]:
%timeit t.a = new_value

69.1 ns ± 0.0331 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6537,  0.6462, -1.1670],
               [-0.1494, -0.1912,  0.7418]]),
    x: Batch(
           c: tensor([[-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6537,  0.6462, -1.1670],
        [-0.1494, -0.1912,  0.7418]])

In [11]:
%timeit b.a

55.7 ns ± 0.0161 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.8373, -1.2267, -0.6559],
               [ 0.7021,  1.4014, -1.8234]]),
    x: Batch(
           c: tensor([[-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119]]),
       ),
)

In [13]:
%timeit b.a = new_value

479 ns ± 0.941 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

833 ns ± 3.55 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 55 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 304 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 243 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4214a71f70>
├── 'a' --> tensor([[[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]],
│           
│                   [[ 0.6537,  0.6462, -1.1670],
│                    [-0.1494, -0.1912,  0.7418]]])
└── 'x' --> <FastTreeValue 0x7f4214a71fa0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 330 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f42c0da1970>
├── 'a' --> tensor([[ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418],
│                   [ 0.6537,  0.6462, -1.1670],
│                   [-0.1494, -0.1912,  0.7418]])
└── 'x' --> <FastTreeValue 0x7f42c0da15e0>
    └── 'c' --> tensor([[-0.1107,  0.4488,  1.0056, -1.7518],
                        [-0.8434,  1.5024, -0.4146,  0.1262],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 36.6 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59.9 µs ± 192 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-0.1107,  0.4488,  1.0056, -1.7518],
                       [-0.8434,  1.5024, -0.4146,  0.1262],
                       [-1.4129,  1.2946, -0.5745, -0.7119]],
              
                      [[-0.1107,  0.4488,  1.0056, -1.7518],
                       [-0.8434,  1.5024, -0.4146,  0.1262],
                       [-1.4129,  1.2946, -0.5745, -0.7119]],
              
                      [[-0.1107,  0.4488,  1.0056, -1.7518],
                       [-0.8434,  1.5024, -0.4146,  0.1262],
                       [-1.4129,  1.2946, -0.5745, -0.7119]],
              
                      [[-0.1107,  0.4488,  1.0056, -1.7518],
                       [-0.8434,  1.5024, -0.4146,  0.1262],
                       [-1.4129,  1.2946, -0.5745, -0.7119]],
              
                      [[-0.1107,  0.4488,  1.0056, -1.7518],
                       [-0.8434,  1.5024, -0.4146,  0.1262],
                       [-1.4129,  1.2946, -0.5745, -0.7119]],

In [26]:
%timeit Batch.stack(batches)

77.3 µs ± 49 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119],
                      [-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119],
                      [-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119],
                      [-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119],
                      [-0.1107,  0.4488,  1.0056, -1.7518],
                      [-0.8434,  1.5024, -0.4146,  0.1262],
                      [-1.4129,  1.2946, -0.5745, -0.7119],
                      [-0.1107,  0.4488,  1.0056, -1.7518],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 354 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 3.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
